
#  Wav2Vec Experiment
https://github.com/pytorch/fairseq/tree/master/examples/wav2vec

## 1 prepare data

In [1]:
!pip install soundfile
!git clone https://github.com/pytorch/fairseq.git

Cloning into 'fairseq'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 19688 (delta 2), reused 7 (delta 2), pack-reused 19677
Receiving objects: 100% (19688/19688), 8.86 MiB | 9.46 MiB/s, done.
Resolving deltas: 100% (14701/14701), done.


In [2]:
%cd fairseq/
!pip install --editable ./

/content/fairseq
Obtaining file:///content/fairseq
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 122kB 14.0MB/s 
     |████████████████████████████████| 71kB 7.7MB/s 
     |████████████████████████████████| 112kB 20.7MB/s 
     |████████████████████████████████| 276kB 11.3MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-cp36-none-any.whl size=141230 sha256=b8b8260ec3bc113d7522e051f55a8afe4a87c4aa0cadb27abfb91f93b3f32157
  Stored in directory: /root/.cache/pip/wheels/e3/e2/fa/b78480b448b8579ddf393bebd3f47ee23aa84c89b6a78285c8
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44619 sha256=b53e09899daa443b643749d831f72ef86bb8600095d7a62893bb1ada7a698cc1
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
S

### Download Data: WSJ

In [3]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
!mkdir /content/WSJ
!cp /content/drive/My\ Drive/project/csr_1_LDC93S6A.tgz /content/WSJ/

In [7]:
%cd /content/WSJ/
!tar -xzf /content/WSJ/csr_1_LDC93S6A.tgz

/content/WSJ


### Download Data: Librispeech

Download data for the first time

In [ ]:
# %cd /content
# !wget http://www.openslr.org/resources/12/train-clean-100.tar.gz
# !wget http://www.openslr.org/resources/12/dev-clean.tar.gz
# !wget http://www.openslr.org/resources/12/test-clean.tar.gz

/content
--2020-11-11 03:49:22--  http://www.openslr.org/resources/12/train-clean-100.tar.gz
Resolving www.openslr.org (www.openslr.org)... 46.101.158.64
Connecting to www.openslr.org (www.openslr.org)|46.101.158.64|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6387309499 (5.9G) [application/x-gzip]
Saving to: ‘train-clean-100.tar.gz’

train-clean-100.tar 100%[===================>]   5.95G  59.4MB/s    in 88s     

2020-11-11 03:50:50 (68.8 MB/s) - ‘train-clean-100.tar.gz’ saved [6387309499/6387309499]



In [ ]:
# copy the dataset to my drive
# !cp train-clean-100.tar.gz /content/drive/My\ Drive/project/
# !cp dev-clean.tar.gz /content/drive/My\ Drive/project/
# !cp test-clean.tar.gz /content/drive/My\ Drive/project/

copy data over drive to local

In [6]:
!cp /content/drive/My\ Drive/project/train-clean-100.tar.gz /content/
!cp /content/drive/My\ Drive/project/dev-clean.tar.gz /content/
!cp /content/drive/My\ Drive/project/test-clean.tar.gz /content/

In [8]:
%cd /content/

#unzip files
!tar -xzf train-clean-100.tar.gz
!tar -xzf dev-clean.tar.gz
!tar -xzf test-clean.tar.gz

%cd /content/
!rm train-clean-100.tar.gz dev-clean.tar.gz test-clean.tar.gz

/content
/content


## 2 pretrain task - Prepare training data manifest:
Given a directory containing wav files to be used for pretraining (we recommend splitting each file into separate file 10 to 30 seconds in length)

In [9]:
!mkdir /content/manifest
!mkdir /content/manifest/train
!mkdir /content/manifest/dev

%cd /content/fairseq
!python examples/wav2vec/wav2vec_manifest.py "/content/LibriSpeech/train-clean-100" \
--dest "/content/manifest/train/" --ext "flac" --valid-percent 0

!python examples/wav2vec/wav2vec_manifest.py "/content/LibriSpeech/dev-clean" \
--dest "/content/manifest/dev/" --ext "flac" --valid-percent 1

!ls -l /content/manifest/train
!ls -l /content/manifest/dev
!mv /content/manifest/dev/valid.tsv /content/manifest/train/

/content/fairseq
total 1104
-rw-r--r-- 1 root root 1122737 Nov 14 07:31 train.tsv
-rw-r--r-- 1 root root      37 Nov 14 07:31 valid.tsv
total 108
-rw-r--r-- 1 root root     31 Nov 14 07:31 train.tsv
-rw-r--r-- 1 root root 106202 Nov 14 07:31 valid.tsv


### Train a wav2vec 2.0 base model (pretrain task):

In [ ]:
!mkdir /content/model
%cd /content/fairseq

/content/fairseq


In [ ]:
!pip install omegaconf
!pip install hydra-core --upgrade

In [ ]:
# --update-freq 64 --distributed-world-size 64 --distributed-port 8888  

In [ ]:
!python train.py "/content/manifest/train" \
--save-dir "/content/model" --fp16 --num-workers 6 --task audio_pretraining --criterion wav2vec --arch wav2vec2 \
--log-keys '["prob_perplexity","code_perplexity","temp"]' --quantize-targets --extractor-mode default \
--conv-feature-layers '[(512, 10, 5)] + [(512, 3, 2)] * 4 + [(512,2,2)] * 2' --final-dim 256 --latent-vars 320 \
--latent-groups 2 --latent-temp '(2,0.5,0.999995)' --infonce --optimizer adam \
--adam-betas '(0.9,0.98)' --adam-eps 1e-06 --lr-scheduler polynomial_decay --total-num-update 400000 \
--lr 0.0005 --warmup-updates 32000 --mask-length 10 --mask-prob 0.65 --mask-selection static --mask-other 0 \
--encoder-layerdrop 0.05 --dropout-input 0.1 --dropout-features 0.1 --feature-grad-mult 0.1 \
--loss-weights '[0.1, 10]' --conv-pos 128 --conv-pos-groups 16 --num-negatives 100 --cross-sample-negatives 0 \
--max-sample-size 250000 --min-sample-size 32000 --dropout 0.1 --attention-dropout 0.1 --weight-decay 0.01 \
--max-tokens 1400000 --max-update 400000 --skip-invalid-size-inputs-valid-test --ddp-backend no_c10d

## 3 Fine-tune task

In [14]:
%cd /content/manifest
!mkdir fine-tune

/content/manifest


### 3.1 generates labels for the Librispeech dataset 
from the tsv file produced by wav2vec_manifest.py

In [46]:
%cd /content/fairseq/examples/wav2vec
!python libri_labels.py /content/manifest/train/train.tsv \
--output-dir /content/manifest/train/ --output-name "train"

!python libri_labels.py /content/manifest/train/valid.tsv \
--output-dir /content/manifest/train/ --output-name "valid"

/content/fairseq/examples/wav2vec


### 3.2 download pretrained wav2vec model

In [20]:
# download pretrained model
!mkdir /content/pretrain_model
%cd /content/pretrain_model
!wget https://dl.fbaipublicfiles.com/fairseq/wav2vec/libri960_big.pt

--2020-11-14 07:41:23--  https://dl.fbaipublicfiles.com/fairseq/wav2vec/libri960_big.pt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.74.142, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3173903620 (3.0G) [application/octet-stream]
Saving to: ‘libri960_big.pt’

libri960_big.pt     100%[===================>]   2.96G  10.8MB/s    in 4m 58s  

2020-11-14 07:46:21 (10.2 MB/s) - ‘libri960_big.pt’ saved [3173903620/3173903620]



### [optional] 3.3 TO-DO: build wav2letter python bindings
https://github.com/facebookresearch/wav2letter/wiki/Building-Python-bindings

### 3.4 download letter vocabulary

In [ ]:
%cd /content/manifest/train
!wget https://dl.fbaipublicfiles.com/fairseq/wav2vec/dict.ltr.txt

### 3.5 run fine-tune


In [ ]:
# save-dir
!mkdir /content/finetune_model

In [49]:
%cd /content/fairseq
!python train.py --distributed-world-size 128 "/content/manifest/train/" --save-dir "/content/finetune_model/" --fp16 \
--post-process letter --valid-subset "/content/manifest/train/valid" --no-epoch-checkpoints --best-checkpoint-metric wer --num-workers 4 \
--max-update 80000 --sentence-avg --task audio_pretraining --arch wav2vec_ctc --w2v-path /content/pretrain_model/libri960_big.pt \
--labels ltr --apply-mask --mask-selection static --mask-other 0 --mask-length 10 --mask-prob 0.5 --layerdrop 0.1 \
--mask-channel-selection static --mask-channel-other 0 --mask-channel-length 64 --mask-channel-prob 0.5 --zero-infinity \
--feature-grad-mult 0.0 --freeze-finetune-updates 10000 --validate-after-updates 10000 --optimizer adam \
--adam-betas '(0.9, 0.98)' --adam-eps 1e-08 --lr 2e-05 --lr-scheduler tri_stage --warmup-steps 8000 --hold-steps 32000 \
--decay-steps 40000 --final-lr-scale 0.05 --final-dropout 0.0 --dropout 0.0 --activation-dropout 0.1 --criterion ctc \
--attention-dropout 0.0 --max-tokens 1280000 --seed 2337 --log-format json --log-interval 500 --ddp-backend no_c10d

/content/fairseq
2020-11-14 08:13:56 | INFO | fairseq_cli.train | {'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 500, 'log_format': 'json', 'tensorboard_logdir': None, 'wandb_project': None, 'seed': 2337, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False}, 'common_eval': {'_name': None, 'path': None, 'post_process': 'letter', 'quiet': False, 'model_overrides': '{}', 'results_path': None}, 'distributed_training': {'_name': None, 'distributed_world_size': 1, 'distributed_rank': 0, 'distributed_backend': 'nccl', 'distributed_init_method': None, 'distributed_port': -1, 'device_id': 0, 'l

## 4 TODO: Evaluation a CTC model:

Fairseq transformer language model used in the wav2vec 2.0 paper

In [ ]:
# language model
%cd /content/LM
!wget https://dl.fbaipublicfiles.com/wav2letter/sota/2019/lm/lm_librispeech_word_transformer.pt
!wget https://dl.fbaipublicfiles.com/wav2letter/sota/2019/lm/lm_librispeech_word_transformer.dict

/content/LM
--2020-11-10 16:03:40--  https://dl.fbaipublicfiles.com/wav2letter/sota/2019/lm/lm_librispeech_word_transformer.pt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 172.67.9.4, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3374337651 (3.1G) [binary/octet-stream]
Saving to: ‘lm_librispeech_word_transformer.pt’

lm_librispeech_word 100%[===================>]   3.14G  23.0MB/s    in 2m 29s  

2020-11-10 16:06:09 (21.6 MB/s) - ‘lm_librispeech_word_transformer.pt’ saved [3374337651/3374337651]



Letter dictionary for pre-trained models can be found here.

In [ ]:
python /content/fairseq/examples/speech_recognition/infer.py /checkpoint/abaevski/data/speech/libri/10h/wav2vec/raw --task audio_pretraining \
--nbest 1 --path /content/model/ --gen-subset "dev_other" --results-path /content/model/ --w2l-decoder kenlm \
--lm-model /path/to/kenlm.bin --lm-weight 2 --world-score -1 --sil-weight 0 --criterion ctc --labels ltr --max-tokens 4000000 \
--post-process letter

SyntaxError: ignored

Wav2Letter

In [ ]:
%cd /content/
!git clone https://github.com/facebookresearch/wav2letter.git

/content
Cloning into 'wav2letter'...
remote: Enumerating objects: 41, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 6215 (delta 11), reused 28 (delta 9), pack-reused 6174
Receiving objects: 100% (6215/6215), 5.98 MiB | 22.69 MiB/s, done.
Resolving deltas: 100% (3990/3990), done.


In [ ]:
%cd wav2letter

/content/wav2letter


In [ ]:
!cd bindings/python
!pip install -e .

/bin/bash: line 0: cd: bindings/python: No such file or directory
ERROR: File "setup.py" not found. Directory cannot be installed in editable mode: /content/wav2letter


In [ ]:
!export KENLM_ROOT_DIR=/root/kenlm && \
cd /root/wav2letter/bindings/python && pip install -e .


/bin/bash: line 0: cd: /root/wav2letter/bindings/python: No such file or directory


In [ ]:
!pip install packaging

In [ ]:
!mkdir build && cd build && cmake .. && make -j8

-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found OpenMP_C: -fopenmp (found version "4.5") 
-- Found OpenMP_CXX: -fopenmp (found version "4.5") 
-- Found OpenMP: TRUE (found version "4.5")  
-- OpenMP found
CMake Error at CMakeLists.txt:25 (find_package):
  By not providing "FindArrayFire.cmake" in CMAKE_MODULE_PATH this project
  has asked CMake to find a package configuration file provided by
  "ArrayFire", but CMake did not find one.

  Cou